In [ ]:
!pip install DMBA mord

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.5 MB/s eta 0:00:00
  Created wheel for mord: filename=mord-0.7-py3-none-any.whl size=9886 sha256=b1e67d5cbaf7db4104b815f7913830f0e1911de0c8b153ebfc843acf00d6b746
  Stored in directory: /root/.cache/pip/wheels/77/00/19/3cea86fbfc737ec4acb515cd94497dcc33f943fa157548b96c
Successfully built mord


In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
from sklearn.feature_selection import RFE
import math
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sys
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import sklearn
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from scipy.stats import uniform
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(sys.version)
print('The scikit-learn version is {}.'.format(sklearn.__version__))

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
The scikit-learn version is 1.5.2.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA = Path('/content/drive/My Drive/Colab Notebooks/Analytics Competition/')

In [ ]:
train_X = pd.read_csv(DATA / 'train_X.csv')
valid_X = pd.read_csv(DATA / 'valid_X.csv')
train_y = pd.read_csv(DATA / 'train_y.csv')
valid_y = pd.read_csv(DATA / 'valid_y.csv')

train_X.head()

,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,DisbursementGross,BalanceGross,GrAppv,Franchise_Dummy,RevLineCr_N,...,State_TN,State_VT,State_WA,State_WI,State_WV,IndustryCode_Other,IndustryCode_Real_Estate,UrbanRural_0,UrbanRural_1,UrbanRural_2
0,1996,84,8,0,0,85000.0,0.0,85000.0,0,1,...,0,0,0,0,0,1,0,1,0,0
1,2002,84,7,1,7,50000.0,0.0,50000.0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2005,68,11,0,0,150000.0,0.0,150000.0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2005,69,15,0,15,11600.0,0.0,11600.0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1999,240,14,7,7,221000.0,0.0,221000.0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
valid_X_scaled = scaler.transform(valid_X)

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(train_X_scaled, train_y.values.ravel())

LinearDiscriminantAnalysis()

In [ ]:
valid_y_pred = lda.predict(valid_X_scaled)

accuracy = accuracy_score(valid_y, valid_y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(valid_y, valid_y_pred))

confusion = confusion_matrix(valid_y, valid_y_pred)
print("Confusion Matrix:")
print(confusion)

Accuracy: 0.832817113655447
              precision    recall  f1-score   support

           0       0.83      0.99      0.91    296702
           1       0.73      0.07      0.13     62964

    accuracy                           0.83    359666
   macro avg       0.78      0.53      0.52    359666
weighted avg       0.82      0.83      0.77    359666

Confusion Matrix:
[[295089   1613]
 [ 58517   4447]]


In [ ]:
f1 = f1_score(valid_y, valid_y_pred)

print(f"F1 Score: {f1}")

F1 Score: 0.12885373203523412


Threshold Optimization for F1 Score:

In [ ]:
thresholds = np.arange(0.1, 1.0, 0.1)
f1_scores = []

for threshold in thresholds:
    valid_y_pred_adjusted = (lda.predict_proba(valid_X_scaled)[:, 1] > threshold).astype(int)
    f1 = f1_score(valid_y, valid_y_pred_adjusted)
    f1_scores.append(f1)

best_threshold = thresholds[np.argmax(f1_scores)]
print(f"Best Threshold: {best_threshold}")

valid_y_pred_final = (lda.predict_proba(valid_X_scaled)[:, 1] > best_threshold).astype(int)

Best Threshold: 0.30000000000000004


In [ ]:
classificationSummary(valid_y, valid_y_pred_final)
f1 = f1_score(valid_y, valid_y_pred_final)
print(f"F1 Score: {f1}")

Confusion Matrix (Accuracy 0.8259)

       Prediction
Actual      0      1
     0 264769  31933
     1  30699  32265
F1 Score: 0.5074629213129709


**Recursive Feature Elimination:**

RFE will help us find the most important features of your model by repeatedly eliminating the least important ones. This can lead to a simpler, less dimensional model and potentially improve accuracy.

In [ ]:
rfe = RFE(estimator=lda, n_features_to_select=5, step=1)
rfe.fit(train_X_scaled, train_y.values.ravel())

selected_features = train_X.columns[rfe.support_]

train_X_selected = train_X[selected_features]
valid_X_selected = valid_X[selected_features]

scaler = StandardScaler()
train_X_scaled_selected = scaler.fit_transform(train_X_selected)
valid_X_scaled_selected = scaler.transform(valid_X_selected)

lda = LinearDiscriminantAnalysis()
lda.fit(train_X_scaled_selected, train_y.values.ravel())


LinearDiscriminantAnalysis()

In [ ]:
valid_y_pred_selected = lda.predict(valid_X_scaled_selected)

print("Classification Summary:")
classificationSummary(valid_y, valid_y_pred_selected)

f1_selected = f1_score(valid_y, valid_y_pred_selected)
print(f"\nF1 Score (with selected features): {f1_selected}")

Classification Summary:
Confusion Matrix (Accuracy 0.8251)

       Prediction
Actual      0      1
     0 296678     24
     1  62880     84

F1 Score (with selected features): 0.002663622526636225


The accuracy saw little to no improvement and the f1score is terrible. This model is a failure.

In [ ]:
best_cutoff_lda_rfe = 0
best_f1_score_lda_rfe = 0

for cutoff in np.arange(0.01, 1, 0.01):
    predicted_probs = lda.predict_proba(valid_X_scaled_selected)[:, 1]
    predictions = (predicted_probs >= cutoff).astype(int)

    f1 = f1_score(valid_y, predictions)

    if f1 > best_f1_score_lda_rfe:
        best_f1_score_lda_rfe = f1
        best_cutoff_lda_rfe = cutoff

print(f"Best cutoff for LDA with RFE (F1-score): {best_cutoff_lda_rfe}")

Best cutoff for LDA with RFE (F1-score): 0.25


In [1]:
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Load your data (replace 'X' and 'y' with actual features and labels)
# Example:
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Example priors
prior_value = [0.6, 0.4]  # Prior distribution for classes

# Linear Discriminant Analysis with prior values
lda = LinearDiscriminantAnalysis(priors=prior_value)
lda.fit(X_train, y_train)

# Predict probabilities
y_scores = lda.decision_function(X_test)  # Function scores
propensities = np.exp(y_scores) / np.sum(np.exp(y_scores), axis=1, keepdims=True)  # Softmax conversion

# New Prior (modify based on domain knowledge)
new_prior = [0.7, 0.3]
lda_new_prior = LinearDiscriminantAnalysis(priors=new_prior)
lda_new_prior.fit(X_train, y_train)

# Adding cost ratio adjustment
cost_ratio = 5  # Adjust according to the problem's misclassification cost
threshold = np.log(cost_ratio) / (1 + np.log(cost_ratio))  # Adjust decision threshold
y_pred_adjusted = (propensities[:, 1] > threshold).astype(int)

# Evaluate model performance
print("Confusion Matrix with Adjusted Threshold:")
print(confusion_matrix(y_test, y_pred_adjusted))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_adjusted))
print("\nAUC-ROC Score:", roc_auc_score(y_test, propensities[:, 1]))

# Visualization or further analysis of propensity scores
propensity_df = pd.DataFrame({"Actual": y_test, "Propensity": propensities[:, 1]})
print(propensity_df.head())


NameError: name 'X_train' is not defined

# Conclusions:

Overall with the accuracy and f1 scores of both the impprved threshold and RFE model, the linear discrimnant analysis does not contend as far as model quality.